### Meghanjali Chennupati (U30308400)
In this assignment, you will use the Dev01 environment to demonstrate:

Launch VSCode and start a terminal inside vscode

Start the Cassandra Cluster the vscode terminal

Create a Jupyter Lab Notebook (in vscode) and use Python to accomplish the following (steps 4 through 9):

Connect to the Cassandra Database

Create a keyspace called bd23

Create a table in this keyspace that will be used to store inventory information. This will include the fields, SKU (which is shown for stock keeping unit), name (short name for the product), description (longer description of the product), and warehouse_num (the warehouse number where the product is stored. NOTE: You must set the appropriate PRIMARY KEY configuration to enable queries described in step #8 (below) to be performed.

Populate several rows of data into your inventory table (create 5 or 6 rows, with at least two rows with a warehouse_num of 123.)

Search for all products located at a given location (warehouse_num 123)

###  Importing all the required packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Creating a keyspace called bd23

In [3]:
session.execute("CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Creating the table inventory using the keyspace bd23.

###  As per question I created inventory table with the following columns SKU, name, description, warehouse_num.

###  Here I am using the primary key as a sku

### As mentioned for me to answer the 7th and 8th step to get the locations of warehouse number 123 then sku will be unique.


In [5]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory ( 
    SKU TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY (SKU)
);
""")


### Load the data

In [7]:
df = pd.read_csv('assignment.csv')
df

,SKU,name,description,warehouse_num,Unnamed: 4
0,'CK100','Prod1','desc1',100,NaN
1,'CK101','Prod2','desc2',100,NaN
2,'CK102','Prod3','desc3',101,NaN
3,'CK103','Prod4','desc4',102,NaN


In [8]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

SKU = 'CK100', name =0, description = 'desc1', warehouse_num = 100
SKU = 'CK101', name =1, description = 'desc2', warehouse_num = 100
SKU = 'CK102', name =2, description = 'desc3', warehouse_num = 101
SKU = 'CK103', name =3, description = 'desc4', warehouse_num = 102


In [9]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ('CK100', 'Prod1', 'desc1', 100);
        

        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ('CK101', 'Prod2', 'desc2', 100);
        

        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ('CK102', 'Prod3', 'desc3', 101);
        

        INSERT INTO bd23.inventory (SKU, name, description, warehouse_num)     
        VALUES ('CK103', 'Prod4', 'desc4', 102);
        


In [10]:
rows = session.execute("select (SKU,name,description,warehouse_num) from bd23.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

SKU=CK101, name=Prod2, description=desc2, warehouse_num=100
SKU=CK100, name=Prod1, description=desc1, warehouse_num=100
SKU=CK103, name=Prod4, description=desc4, warehouse_num=102
SKU=CK102, name=Prod3, description=desc3, warehouse_num=101


###  Search for all products located at a given location (warehouse_num 123)

In [12]:
rows = session.execute("select (SKU,name,description,warehouse_num) from bd23.inventory where warehouse_num=100 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

CK101, Prod2, desc2, 100
CK100, Prod1, desc1, 100


### Analysis:

1. If we observe the above query as per question i extracted the information of goods whose warehouse_num is 123.
2. As mentioned in question to take two rows of warehouse location 123 i did.
3. How ever , since it is point query allow filteirng is working how ever for range queries indexing will help.
4. How ever, we can also create index on the warehouse_num and then no need to specify the allow filtering and we can extract the information

In [13]:
session.execute("""
        CREATE INDEX IF NOT EXISTS idx_warehouse_num ON bd23.inventory(warehouse_num);
""")

In [14]:
rows = session.execute("select (SKU,name,description,warehouse_num) from bd23.inventory where warehouse_num=123")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

### As i mentioned I used the index and extracted the records of products whose warehouse_num is 123.